In [1]:
import tensorflow as tf

In [2]:
import numpy as np
from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, Dropout
from tensorflow.keras.models import Model

Load in the data

In [3]:
fashion_mnist = tf.keras.datasets.fashion_mnist

(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
print("x_train.shape:",x_train.shape)

4422102/4422102 [==============================] - 0s 0us/step
x_train.shape: (60000, 28, 28)


The data is only 2D

Convolution expects height x widht x color (3D)

In [4]:
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print(x_train.shape)

(60000, 28, 28, 1)


Number of classes

In [5]:
k = len(set(y_train))
print("number of classes:", k)

number of classes: 10


Build the model using the funcional API

In [10]:
def create_model():
  i = Input(x_train[0].shape)
  x = Conv2D(32, (3, 3), strides=2, activation='relu')(i)
  x = Conv2D(64, (3, 3), strides=2, activation='relu')(x)
  x = Conv2D(128, (3, 3), strides=2, activation='relu')(x)
  x = Flatten()(x)
  x = Dropout(0.2)(x)
  x = Dense(512, activation='relu')(x)
  x = Dropout(0.2)(x)
  x = Dense(k, activation='softmax')(x)

  model = Model(i, x)
  return model

Strategy

NOTE: use GPU

In [7]:
strategy = tf.distribute.MirroredStrategy()

In [8]:
print("Number of devices: ", strategy.num_replicas_in_sync)

Number of devices:  1


In [11]:
with strategy.scope():
  model = create_model()
  model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

Fit using large batch_size and Mirrored Strategy

In [13]:
r = model.fit(x_train, y_train, validation_data=(x_test, y_test), batch_size=128, epochs=15)

Epoch 1/15
469/469 [==============================] - 5s 9ms/step - loss: 0.0828 - accuracy: 0.9691 - val_loss: 0.3675 - val_accuracy: 0.9097
Epoch 2/15
469/469 [==============================] - 3s 7ms/step - loss: 0.0687 - accuracy: 0.9738 - val_loss: 0.3868 - val_accuracy: 0.9077
Epoch 3/15
469/469 [==============================] - 4s 8ms/step - loss: 0.0632 - accuracy: 0.9761 - val_loss: 0.3900 - val_accuracy: 0.9056
Epoch 4/15
469/469 [==============================] - 4s 8ms/step - loss: 0.0606 - accuracy: 0.9767 - val_loss: 0.4160 - val_accuracy: 0.9048
Epoch 5/15
469/469 [==============================] - 4s 8ms/step - loss: 0.0586 - accuracy: 0.9775 - val_loss: 0.4247 - val_accuracy: 0.9059
Epoch 6/15
469/469 [==============================] - 3s 7ms/step - loss: 0.0572 - accuracy: 0.9780 - val_loss: 0.4226 - val_accuracy: 0.9049
Epoch 7/15
469/469 [==============================] - 3s 7ms/step - loss: 0.0551 - accuracy: 0.9790 - val_loss: 0.4486 - val_accuracy: 0.9046
Epoch 

In [ ]:
model2 = create_model()
model2.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
r = model2.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=15)

Epoch 1/15
1875/1875 [==============================] - 38s 20ms/step - loss: 0.5225 - accuracy: 0.8059 - val_loss: 0.4090 - val_accuracy: 0.8508
Epoch 2/15
1875/1875 [==============================] - 34s 18ms/step - loss: 0.3662 - accuracy: 0.8626 - val_loss: 0.3479 - val_accuracy: 0.8689
Epoch 3/15
1875/1875 [==============================] - 36s 19ms/step - loss: 0.3174 - accuracy: 0.8812 - val_loss: 0.3358 - val_accuracy: 0.8726
Epoch 4/15
1875/1875 [==============================] - 40s 21ms/step - loss: 0.2815 - accuracy: 0.8944 - val_loss: 0.2959 - val_accuracy: 0.8900
Epoch 5/15
1875/1875 [==============================] - 46s 24ms/step - loss: 0.2556 - accuracy: 0.9028 - val_loss: 0.3018 - val_accuracy: 0.8881
Epoch 6/15
1875/1875 [==============================] - 44s 24ms/step - loss: 0.2323 - accuracy: 0.9117 - val_loss: 0.3001 - val_accuracy: 0.8931
Epoch 7/15
1875/1875 [==============================] - 42s 22ms/step - loss: 0.2123 - accuracy: 0.9188 - val_loss: 0.3019 -